In [1]:
%load_ext autoreload
%autoreload 2
is_debug = 1
import os
import re
import gc
import sys
import glob
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

#========================================================================
# original library 
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
sys.path.append(f"{HOME}/kaggle/data_analysis/model/")
sys.path.append(f"../py")
import MS_utils
from params_MS import params_lgb
import utils, ml_utils
from utils import logger_func
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()
#========================================================================

"""
argv[1]: comment
argv[2]: feature_key
"""
# Basic Args
seed = 1208
set_type = 'all'
fold_n = 4
key, raw_target, ignore_list = MS_utils.get_basic_var()
comment = sys.argv[1]

# Base
vi_col = 'f000_AvSigVersion'
base_path = '../input/base_exclude*'
base_path = '../input/base_Av*'
#  base_path = '../input/base_group*'
base = utils.read_df_pkl(base_path)

#========================================================================
# Make Validation
cv = KFold(n_splits=fold_n, shuffle=False, random_state=seed)
if is_debug:
    base_train = base[base[raw_target].isnull()].head(10000).sort_values(by=vi_col)
else:
    base_train = base[base[raw_target].isnull()].sort_values(by=vi_col)

kfold = list(cv.split(base_train[[key, 'country_group']], base_train[vi_col]))
del base_train
gc.collect()
base = base[[key, raw_target]]
#========================================================================

#  train, test = MS_utils.get_dataset(base=base)
test, train = MS_utils.get_dataset(base=base)

if is_debug:
    train = train.head(10000)
    test = test.head(500)
print(train.shape, test.shape)

#========================================================================
# Categorical Encode
ignore_list = [key, raw_target]
cat_cols = utils.get_categorical_features(df=train, ignore_list=ignore_list)
print(f"Categorical: {cat_cols}")

#Fit LabelEncoder
for col in cat_cols:
    # 最も頻度の多いカテゴリでimpute
    max_freq = train[col].mode()
    train[col].fillna(max_freq, inplace=True)
    test[col].fillna(max_freq, inplace=True)
    le = LabelEncoder().fit(pd.concat([train[col], test[col]], axis=0).value_counts().index.tolist())
    train[col] = le.transform(train[col])
    test[col]  = le.transform(test[col])
#========================================================================

2019-03-08 23:12:57,337 utils 346 [INFO]    [logger_func] start 
100%|██████████| 3/3 [00:01<00:00,  2.84it/s]


(16774736, 2)
(8921483, 83) (7853253, 83)
Categorical: ['f000_EngineVersion', 'f000_Census_OSBranch', 'f000_Census_GenuineStateName', 'f000_Census_OSArchitecture', 'f000_Census_OSVersion', 'f000_Platform', 'f000_Census_PrimaryDiskTypeName', 'f000_ProductName', 'f000_SkuEdition', 'f000_SmartScreen', 'f000_PuaMode', 'f000_Census_ActivationChannel', 'f000_Census_DeviceFamily', 'f000_Census_OSSkuName', 'f000_OsVer', 'f000_Census_ChassisTypeName', 'f000_Census_InternalBatteryType', 'f000_OsPlatformSubRelease', 'f000_Census_ProcessorClass', 'f000_Census_PowerPlatformRoleName', 'f000_Census_OSInstallTypeName', 'f000_Census_OSWUAutoUpdateOptionsName', 'f000_Census_OSEdition', 'f000_Census_FlightRing', 'f000_AppVersion', 'f000_Census_MDC2FormFactor', 'f000_Processor', 'f000_OsBuildLab']
(8921483, 83) (7853253, 83)
(10000, 83) (500, 83)
Categorical: ['f000_AvSigVersion']


In [5]:
model_type_list = ['lgb', 'rmf', 'lgr']
model_type_list = ['lgb']
model_type = 'lgb'
metric = 'auc'
metric = 'rmse'

target_list = [
    'f000_SmartScreen'
    ,'f000_AvSigVersion'
    ,'f000_EngineVersion'
    ,'f000_AppVersion'
    ,'f000_Census_OSVersion'
    ,'f000_AVProductStatesIdentifier'
    ,'f000_CountryIdentifier'
    ,'f000_AVProductsInstalled'
    ]

for target in target_list:

    feim_list = []
    score_list = []
    oof_pred = np.zeros(len(train))
    y_test = np.zeros(len(test))

    Y = train[target]
    ignore_list = [key, raw_target, target, 'country_group']
    use_cols = [col for col in train.columns if col not in ignore_list]
    x_test = test[use_cols]
    start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

    if model_type=='lgb':
        params = params_lgb()
        #  params['num_leaves'] = 4
        params['num_threads'] = 36
        try:
            #  params['learning_rate'] = float(sys.argv[1])
            params['learning_rate'] = 0.05
        except IndexError:
            pass
    else:
        params = {}

    logger.info(f"{model_type} Train Start!!")

    for num_fold, (trn_idx, val_idx) in enumerate(kfold):
        params['objective'] = 'regression'
        x_train, y_train = train[use_cols].iloc[trn_idx, :], Y.iloc[trn_idx]
        x_val, y_val = train[use_cols].iloc[val_idx, :], Y.iloc[val_idx]

        logger.info(f"Fold{num_fold} | Train:{x_train.shape} | Valid:{x_val.shape}")

        #  score, tmp_oof, tmp_pred, feim = ml_utils.Classifier(
        score, tmp_oof, tmp_pred, feim = ml_utils.Regressor(
            model_type=model_type
            , x_train=x_train
            , y_train=y_train
            , x_val=x_val
            , y_val=y_val
            , x_test=x_test
            , params=params
            , seed=seed
            , get_score=metric
        )
        feim_list.append(feim.set_index('feature').rename(columns={'importance':f'imp_{num_fold}'}))

        logger.info(f"Fold{num_fold} CV: {score}")
        score_list.append(score)
        oof_pred[val_idx] = tmp_oof
        y_test += tmp_pred

    feim = pd.concat(feim_list, axis=1)
    feim_cols = [col for col in feim.columns if col.count('imp_')]
    feim['importance'] = feim[feim_cols].mean(axis=1)
    feim.drop(feim_cols, axis=1, inplace=True)
    feim.sort_values(by='importance', ascending=False, inplace=True)

    cv_score = np.mean(score_list)
    logger.info(f'''
    #========================================================================
    # Model: {model_type}
    # CV   : {cv_score}
    #========================================================================''')

    y_test /= (num_fold+1)

    pred_col = 'prediction'
    train[pred_col] = oof_pred
    test[pred_col] = y_test
    stack_cols = [key, target, pred_col]

    df_stack = pd.concat([train[stack_cols], test[stack_cols]], ignore_index=True, axis=0)

    #========================================================================
    # Saving
    feim.to_csv(f'../valid/{start_time[4:12]}_{model_type}_SET-{set_type}_feat{len(x_train.columns)}_{comment}_CV{str(cv_score)[:7]}_LB.csv', index=True)
    utils.to_pkl_gzip(obj=df_stack, path=f'../stack/{start_time[4:12]}_{model_type}_SET-{set_type}_feat{len(x_train.columns)}_{comment}_CV{str(cv_score)[:7]}_LB')
    #========================================================================

2019-03-08 23:17:48,747 utils 43 [INFO]    [<module>] lgb Train Start!! 
2019-03-08 23:17:48,757 utils 50 [INFO]    [<module>] Fold0 | Train:(7500, 80) | Valid:(2500, 80) 


Training until validation scores don't improve for 150 rounds.


2019-03-08 23:17:49,610 utils 66 [INFO]    [<module>] Fold0 CV: 2.224527614451031 
2019-03-08 23:17:49,624 utils 50 [INFO]    [<module>] Fold1 | Train:(7500, 80) | Valid:(2500, 80) 


[200]	valid_0's l2: 5.03831
Early stopping, best iteration is:
[69]	valid_0's l2: 4.94852

        # R2 Score: 0.03273590550133032
        
Training until validation scores don't improve for 150 rounds.


2019-03-08 23:17:50,403 utils 66 [INFO]    [<module>] Fold1 CV: 2.235360261322957 
2019-03-08 23:17:50,414 utils 50 [INFO]    [<module>] Fold2 | Train:(7500, 80) | Valid:(2500, 80) 


...

2019-03-08 23:19:03,575 utils 66 [INFO]    [<module>] Fold2 CV: 3.7208004751422803 
2019-03-08 23:19:03,588 utils 50 [INFO]    [<module>] Fold3 | Train:(7500, 81) | Valid:(2500, 81) 



        # R2 Score: 0.9989983520544132
        
Training until validation scores don't improve for 150 rounds.
[200]	valid_0's l2: 16.862
[400]	valid_0's l2: 12.4169
[600]	valid_0's l2: 11.1936
[800]	valid_0's l2: 10.8596
[1000]	valid_0's l2: 10.6584
[1200]	valid_0's l2: 10.5709
[1400]	valid_0's l2: 10.5014
[1600]	valid_0's l2: 10.4631
[1800]	valid_0's l2: 10.4415
[2000]	valid_0's l2: 10.43
[2200]	valid_0's l2: 10.4242
[2400]	valid_0's l2: 10.4184
[2600]	valid_0's l2: 10.4157
[2800]	valid_0's l2: 10.4137
[3000]	valid_0's l2: 10.4112
[3200]	valid_0's l2: 10.4095
[3400]	valid_0's l2: 10.4084
[3600]	valid_0's l2: 10.4075
[3800]	valid_0's l2: 10.4073
[4000]	valid_0's l2: 10.4068
[4200]	valid_0's l2: 10.4066
[4400]	valid_0's l2: 10.4064
[4600]	valid_0's l2: 10.4064
[4800]	valid_0's l2: 10.4063
[5000]	valid_0's l2: 10.4063
[5200]	valid_0's l2: 10.4062
[5400]	valid_0's l2: 10.4062
[5600]	valid_0's l2: 10.4062
[5800]	valid_0's l2: 10.4061
[6000]	valid_0's l2: 10.4061
[6200]	valid_0's l2: 10.4

2019-03-08 23:19:15,105 utils 66 [INFO]    [<module>] Fold3 CV: 3.2258521813080154 
2019-03-08 23:19:15,112 utils 82 [INFO]    [<module>] 
    #========================================================================
    # Model: lgb
    # CV   : 4.048228570141893
    #======================================================================== 
2019-03-08 23:19:15,186 utils 43 [INFO]    [<module>] lgb Train Start!! 
2019-03-08 23:19:15,202 utils 50 [INFO]    [<module>] Fold0 | Train:(7500, 81) | Valid:(2500, 81) 



        # R2 Score: 0.9992326156457059
        
Training until validation scores don't improve for 150 rounds.


TypeError: 'NoneType' object is not iterable

In [16]:
x_train.head()

,f000_EngineVersion,f000_Census_TotalPhysicalRAM,f000_IsBeta,f000_RtpStateBitfield,f000_Census_OSBranch,f000_Census_IsSecureBootEnabled,f000_Census_GenuineStateName,f000_Census_ProcessorCoreCount,f000_Census_OSArchitecture,f000_AVProductsEnabled,...,f000_Census_ThresholdOptIn,f000_UacLuaenable,f000_Census_MDC2FormFactor,f000_Census_SystemVolumeTotalCapacity,f000_Wdft_RegionIdentifier,f000_Census_IsTouchEnabled,f000_Processor,f000_Census_IsVirtualDevice,f000_OsBuildLab,prediction
2500,69,8192.0,0,7.0,30,0,1,4.0,0,1.0,...,0.0,1.0,2,476438.0,10.0,0,1,0.0,46,40.042642
2501,72,8192.0,0,7.0,12,1,1,4.0,0,1.0,...,NaN,1.0,8,119434.0,3.0,0,1,0.0,306,430.783805
2502,71,16384.0,0,7.0,12,0,1,4.0,0,1.0,...,0.0,1.0,2,2861021.0,15.0,0,1,0.0,306,428.360980
2503,69,4096.0,0,7.0,10,1,1,4.0,0,1.0,...,NaN,1.0,8,459239.0,11.0,0,1,0.0,292,363.105245
2504,69,12288.0,0,7.0,10,1,1,4.0,0,1.0,...,NaN,1.0,8,468133.0,11.0,0,1,0.0,292,363.587898
